In [1]:
#fine tuning


from lightfm import LightFM
from lightfm.evaluation import precision_at_k
import itertools

# Define fine-tuning search grid
param_grid = {
    'no_components': [20, 40],           # Latent features
    'learning_rate': [0.01, 0.05],       # Step size
    'loss': ['warp', 'bpr']              # Ranking loss functions
}

# Generate all hyperparameter combinations
param_combinations = list(itertools.product(
    param_grid['no_components'],
    param_grid['learning_rate'],
    param_grid['loss']
))

# Initialize trackers
best_score = 0
best_model = None
best_params = None
scores_log = []

print("🔍 Fine-Tuning LightFM...\n")

for no_components, learning_rate, loss in param_combinations:
    model = LightFM(no_components=no_components, learning_rate=learning_rate, loss=loss)
    model.fit(interactions, epochs=50, num_threads=2)  # Train longer

    precision = precision_at_k(model, interactions, k=5).mean()
    scores_log.append((no_components, learning_rate, loss, precision))

    print(f"🔧 Params: components={no_components}, lr={learning_rate}, loss={loss} ➤ Precision@5: {precision:.4f}")

    if precision > best_score:
        best_score = precision
        best_model = model
        best_params = (no_components, learning_rate, loss)

# Show final results
print("\n✅ Best Model Found:")
print(f"Components     : {best_params[0]}")
print(f"Learning Rate  : {best_params[1]}")
print(f"Loss Function  : {best_params[2]}")
print(f"Best Precision@5: {best_score:.4f}")


ModuleNotFoundError: No module named 'lightfm'

In [ ]:
# fine tuning 2
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
import itertools
import random
import numpy as np

# 🧪 STEP 1: Stretch ratings to range 0.3–1.0 for better learning
df_filtered['nutrient_rank'] = df_filtered['nutrient_score'].rank(ascending=True)
df_filtered['rating'] = 0.3 + (0.7 * (1 - df_filtered['nutrient_rank'] / df_filtered['nutrient_rank'].max()))

# 🎯 STEP 2: Limit to top 1000 nutrient-rich recipes
df_filtered = df_filtered.sort_values(by='rating', ascending=False).head(1000)

# 👤 STEP 3: Simulate user interactions
df_filtered['user_id'] = [random.choice(['user1', 'user2', 'user3', 'user4', 'user5']) for _ in range(len(df_filtered))]

# 🔗 STEP 4: Build dataset and interactions
from lightfm.data import Dataset
dataset = Dataset()
dataset.fit(df_filtered['user_id'], df_filtered['recipe_name'])
(interactions, weights) = dataset.build_interactions([
    (row['user_id'], row['recipe_name'], row['rating']) for _, row in df_filtered.iterrows()
])
user_id_map, _, item_id_map, _ = dataset.mapping()
item_labels = list(item_id_map.keys())

# 🔍 STEP 5: Fine-Tuning LightFM
param_grid = {
    'no_components': [20, 40],
    'learning_rate': [0.01, 0.05],
    'loss': ['warp', 'bpr']
}
param_combinations = list(itertools.product(
    param_grid['no_components'],
    param_grid['learning_rate'],
    param_grid['loss']
))

best_score = 0
best_model = None
best_params = None

print("🔍 Fine-Tuning LightFM...\n")

for no_components, learning_rate, loss in param_combinations:
    model = LightFM(no_components=no_components, learning_rate=learning_rate, loss=loss)
    model.fit(interactions, epochs=50, num_threads=2)  # ⏳ train longer

    precision = precision_at_k(model, interactions, k=5).mean()
    print(f"✅ Params → components={no_components}, lr={learning_rate}, loss={loss} ➤ Precision@5: {precision:.4f}")

    if precision > best_score:
        best_score = precision
        best_model = model
        best_params = (no_components, learning_rate, loss)

# 🏁 STEP 6: Use the best model for recommendations
print("\n🎯 Best Model Summary:")
print(f"Components:     {best_params[0]}")
print(f"Learning Rate:  {best_params[1]}")
print(f"Loss Function:  {best_params[2]}")
print(f"Best Precision@5: {best_score:.4f}")

def recommend_top_n(user, model, item_labels, n=5):
    user_index = user_id_map[user]
    scores = model.predict(user_index, np.arange(len(item_labels)))
    top_indices = np.argsort(-scores)[:n]
    return [(item_labels[i], round(scores[i], 2)) for i in top_indices]

# 🔮 Final recommendations
print("\n📌 Top 5 Recommendations for user1:")
print(recommend_top_n('user1', best_model, item_labels))

print("\n📌 Top 5 Recommendations for user2:")
print(recommend_top_n('user2', best_model, item_labels))


🔍 Fine-Tuning LightFM...

✅ Params → components=20, lr=0.01, loss=warp ➤ Precision@5: 1.0000
✅ Params → components=20, lr=0.01, loss=bpr ➤ Precision@5: 1.0000
✅ Params → components=20, lr=0.05, loss=warp ➤ Precision@5: 1.0000
✅ Params → components=20, lr=0.05, loss=bpr ➤ Precision@5: 1.0000
✅ Params → components=40, lr=0.01, loss=warp ➤ Precision@5: 1.0000
✅ Params → components=40, lr=0.01, loss=bpr ➤ Precision@5: 1.0000
✅ Params → components=40, lr=0.05, loss=warp ➤ Precision@5: 1.0000
✅ Params → components=40, lr=0.05, loss=bpr ➤ Precision@5: 1.0000

🎯 Best Model Summary:
Components:     20
Learning Rate:  0.01
Loss Function:  warp
Best Precision@5: 1.0000

📌 Top 5 Recommendations for user1:
[('chicken crescent squares', np.float32(-0.01)), ('mushroom and grilled onion quiche', np.float32(-0.03)), ('tart cherry crisp with oaty crumble topping', np.float32(-0.03)), ('paleo chicken bacon strawberry salad with orange balsamic recipes', np.float32(-0.03)), ('chai latte pots de creme', np